Step 1:Import Required Libraries (Code Cell)

In [12]:
# -----------------------------
# 1️⃣ Data Manipulation & Analysis
# -----------------------------
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import warnings
warnings.filterwarnings("ignore")

# -----------------------------
# 2️⃣ Visualization
# -----------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# -----------------------------
# 3️⃣ LSTM / Neural Network
# -----------------------------
from tensorflow.keras.models import load_model
import joblib  # for scalers, if needed

# -----------------------------
# 4️⃣ Connect project src modules
# -----------------------------
import sys
# Add your src folder to Python path
sys.path.append("D:/Personal/KAIM-10 Academy/Week 9/Project/portfolio-optimization-week9/src")

# Import project modules
import processed_data
import forecasting
import visualization
import evaluation
import utils
import modeling

print("✅ Project src path successfully found and modules imported")
# Load processed data
processed_file = Path("D:/Personal/KAIM-10 Academy/Week 9/Project/portfolio-optimization-week9/data/processed/all_assets_clean.csv")
df = pd.read_csv(processed_file, parse_dates=["Date"])

# Filter Tesla and rename price column to 'Close' for consistency
df_tsla = df[df["Ticker"] == "TSLA"][["Date", "Price"]].rename(columns={"Price": "Close"})

# Sort by Date and remove duplicates
df_tsla = df_tsla.sort_values("Date").drop_duplicates(subset="Date", keep="first").reset_index(drop=True)

# Check first rows
df_tsla.head()

✅ Project src path successfully found and modules imported


,Date,Close
0,2015-01-02,0.007355
1,2015-01-05,0.006459
2,2015-01-06,0.006574
3,2015-01-07,0.006542
4,2015-01-08,0.006510


In [13]:
# Train/Test split (same as Task 2)
train, test = utils.train_test_split_time(df_tsla, target_col="Close", train_end_date="2024-12-31")

Step 2: Train ARIMA

In [14]:
tsla_series = train.set_index("Date")["Close"].asfreq('B').ffill()
arima_model, order, seasonal_order = modeling.fit_arima(tsla_series)
forecast_arima, _ = modeling.forecast_sarimax(tsla_series, order, seasonal_order, steps=len(test))


D:\Personal\KAIM-10 Academy\Week 9\Project\portfolio-optimization-week9\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Step 3: Train LTSM

In [15]:
# Scale data
train_scaled, test_scaled, scaler = utils.scale_data(train, test, col="Close")

X_train, y_train = utils.create_lstm_sequences(train_scaled, seq_length=60)
X_test, y_test = utils.create_lstm_sequences(np.concatenate([train_scaled[-60:], test_scaled]), seq_length=60)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build & train
lstm_model = modeling.build_lstm(input_shape=(X_train.shape[1], X_train.shape[2]))
lstm_model, history = modeling.train_lstm(lstm_model, X_train, y_train, epochs=50, batch_size=32)

# Forecast
y_pred_lstm = modeling.forecast_lstm(lstm_model, X_test)
y_pred_lstm = scaler.inverse_transform(y_pred_lstm)


Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 0.0231 - val_loss: 0.0026
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 7.9928e-04 - val_loss: 0.0018
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 6.3031e-04 - val_loss: 0.0023
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 5.9928e-04 - val_loss: 0.0017
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 5.4459e-04 - val_loss: 0.0016
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 5.0098e-04 - val_loss: 0.0016
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 5.2602e-04 - val_loss: 0.0015
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 4.6924e-04 - val_loss: 0.0014
Epoch 9/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 4.6651e-04 - val_loss: 0.0014
Epoch 10/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 4.3769e-04 - val_loss: 0.0015
Epoch 11/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 4.2947e-04 - val_loss: 0.0013
Epoch 12/50
70/70 ━━━━━

Evaluate & choose best model

In [17]:
arima_mae = evaluation.compute_mae(test["Close"].values, forecast_arima)
arima_rmse = evaluation.compute_rmse(test["Close"].values, forecast_arima)
arima_mape = evaluation.compute_mape(test["Close"].values, forecast_arima)

lstm_mae = evaluation.compute_mae(test["Close"].values, y_pred_lstm.flatten())
lstm_rmse = evaluation.compute_rmse(test["Close"].values, y_pred_lstm.flatten())
lstm_mape = evaluation.compute_mape(test["Close"].values, y_pred_lstm.flatten())

results = {
    "ARIMA": {"MAE": arima_mae, "RMSE": arima_rmse, "MAPE": arima_mape},
    "LSTM": {"MAE": lstm_mae, "RMSE": lstm_rmse, "MAPE": lstm_mape}
}

comparison_df = evaluation.compare_models(results)
print(comparison_df)

            MAE      RMSE       MAPE
ARIMA  0.101372  0.120970  23.313764
LSTM   0.017845  0.022788   3.633614


Save the Model

In [22]:
lstm_model_path = model_folder / "lstm_tsla_model.keras"
lstm_model.save(lstm_model_path)

print(f"✅ LSTM model saved in native Keras format at: {lstm_model_path}")

✅ LSTM model saved in native Keras format at: D:\Personal\KAIM-10 Academy\Week 9\Project\portfolio-optimization-week9\models\lstm_tsla_model.keras


In [23]:
# -----------------------------
# Recreate scaler from training data
# -----------------------------
from pathlib import Path
import pandas as pd
import joblib
from sklearn.preprocessing import MinMaxScaler

# Load processed data
processed_file = Path(
    "D:/Personal/KAIM-10 Academy/Week 9/Project/portfolio-optimization-week9/data/processed/all_assets_clean.csv"
)

df = pd.read_csv(processed_file, parse_dates=["Date"])

# Filter Tesla & rename Price -> Close (important!)
df_tsla = (
    df[df["Ticker"] == "TSLA"][["Date", "Price"]]
    .rename(columns={"Price": "Close"})
    .sort_values("Date")
    .drop_duplicates(subset="Date")
    .reset_index(drop=True)
)

# Train/Test split (same cutoff as Task 2)
train = df_tsla[df_tsla["Date"] <= "2024-12-31"]

# Fit scaler ONLY on training data
scaler = MinMaxScaler()
scaler.fit(train[["Close"]])

# Save scaler
scaler_path = Path(
    "D:/Personal/KAIM-10 Academy/Week 9/Project/portfolio-optimization-week9/models/tsla_scaler.pkl"
)
joblib.dump(scaler, scaler_path)

print("✅ Scaler recreated and saved successfully")

✅ Scaler recreated and saved successfully
